In [1]:
%run env_large.ipynb


pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def train(num_episodes, env, agent, vis=False):
    steps_per_e = np.zeros(num_episodes)
    r_per_e = np.zeros(num_episodes)
    for episode in tqdm(range(num_episodes)):
        terminal = False
        s = env.reset()
        r_t = 0
        t = 0
        while not terminal:
            t += 1
            a = agent.choose_action(s)
            sp, r, terminal = env.step(a)
            Q, s_count, sa_count = agent.learn(s, a, r, sp)
            r_t += r
            s = sp

            if vis:
                cont = env.render()
                if not cont or terminal:
                    pg.quit()
                    vis = False

        steps_per_e[episode] = t
        r_per_e[episode] = r_t

    return Q, s_count, sa_count, steps_per_e, r_per_e


In [3]:
def play(env, Q):
    pg.init()
    agent = Agent(n_actions, n_dims, gamma, alpha_p, Q)
    terminal = False
    s = env.reset()
    while not terminal:
        a = agent.choose_action_egreedy(s)
        sp, r, terminal = env.step(a)
        cont = env.render()
        if cont is False:
            break
    pg.quit()

In [4]:
def load_weights(idx, chunk):
    post = f"{(idx + 1) * chunk}.npy"
    Q = np.load(f"Q_{post}")
    s_count = np.load(f"s_count_{post}")
    sa_count = np.load(f"sa_count_{post}")
    return Q, s_count, sa_count
    
def save_weights(dict_, idx, chunk):
    for name, val in dict_.items():
        np.save(f"{name}_{(idx + 1) * chunk}", val)


In [5]:
if __name__ == "__main__":
    gamma = 0.990
    alpha_p = 0.999
    env = Env()
    offset = 1_000
    num_episodes = 1_000
    try:
        Q = np.load(f"Q_{offset}.npy")
        s_count = np.load(f"s_count_{offset}.npy")
        sa_count = np.load(f"sa_count_{offset}.npy")
        print("Imported Weights!")
    except FileNotFoundError:
        Q, s_count, sa_count = None, None, None


In [6]:
# agent = Agent(n_actions, n_dims, gamma, alpha_p, Q, s_count, sa_count)    
# Q, s_count, sa_count, steps_per_e, r_per_e = train(num_episodes=num_episodes, env=env, agent=agent, vis=True)

# np.save(f"Q_{num_episodes + offset}", Q)
# np.save(f"s_count_{num_episodes + offset}", s_count)
# np.save(f"sa_count_{num_episodes + offset}", sa_count)

# plt.plot(steps_per_e)
# # plt.plot(r_per_e)


In [9]:
env = Env(n_dims, n_states, actions)
play(env, Q)